In [6]:
%matplotlib notebook
import numpy as np
import cv2
from utils import imshow

In [7]:
cap = cv2.VideoCapture('../nocutcorners.mp4')

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# imshow("ff",old_frame, size=6)

p0 = np.float32([[[153, 124]]
     ,[[154, 341]]
     ,[[472, 128]]
     ,[[483,333]]])


#TODO: aspect ratio
size = 800
p0.shape

(4, 1, 2)

In [8]:
pts_warped = np.float32([[0,0],[1500,0],[0,1000],[1500,1000]])

while(1):
    ret,frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    M = cv2.getPerspectiveTransform(p1,pts_warped)

    dst = cv2.warpPerspective(frame,M,(1500,1000))
    cv2.imshow('frame', frame)
    cv2.imshow('rectified',dst)
    blurred = cv2.GaussianBlur(frame_gray, (5, 5), 0)
    thresh = cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY)[1]
    cv2.imshow('binary', thresh)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()